# Platform Utilities

> Cross-platform utilities for process management, path handling, and system detection

In [ ]:
#| default_exp core.platform

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import platform
import subprocess
import json
import tarfile
import tempfile
import shutil
import urllib.request
from pathlib import Path
from typing import Dict, Any, Optional, List, TYPE_CHECKING

from cjm_plugin_system.core.config import CJMConfig

This module provides cross-platform utilities to support Linux, macOS, and Windows:

| Category | Functions |
|----------|----------|
| **Detection** | `is_windows()`, `is_macos()`, `is_linux()`, `is_apple_silicon()`, `get_current_platform()` |
| **Paths** | `get_python_in_env()` |
| **Process** | `get_popen_isolation_kwargs()`, `terminate_process()`, `terminate_self()` |
| **Shell** | `run_shell_command()`, `conda_env_exists()` |
| **Conda/Micromamba** | `get_micromamba_download_url()`, `download_micromamba()`, `get_conda_command()`, `build_conda_command()`, `ensure_runtime_available()` |

## Platform Detection

Functions to detect the current operating system and architecture.

In [ ]:
#| export
def is_windows() -> bool:
    """Check if running on Windows."""
    return platform.system() == "Windows"


def is_macos() -> bool:
    """Check if running on macOS."""
    return platform.system() == "Darwin"


def is_linux() -> bool:
    """Check if running on Linux."""
    return platform.system() == "Linux"


def is_apple_silicon() -> bool:
    """Check if running on Apple Silicon Mac (for MPS detection)."""
    return is_macos() and platform.machine() == "arm64"

In [ ]:
# Test detection functions
print(f"is_windows(): {is_windows()}")
print(f"is_macos(): {is_macos()}")
print(f"is_linux(): {is_linux()}")
print(f"is_apple_silicon(): {is_apple_silicon()}")

# Exactly one of these should be True
assert sum([is_windows(), is_macos(), is_linux()]) == 1

is_windows(): False
is_macos(): False
is_linux(): True
is_apple_silicon(): False


In [ ]:
#| export
def get_current_platform() -> str:
    """Get current platform string for manifest filtering.
    
    Returns strings like 'linux-x64', 'darwin-arm64', 'win-x64'.
    """
    system = platform.system().lower()
    machine = platform.machine().lower()
    
    # Normalize system names
    if system == "darwin":
        pass  # Keep as darwin
    elif system == "windows":
        system = "win"
    
    # Normalize architecture
    if machine in ("x86_64", "amd64"):
        arch = "x64"
    elif machine in ("arm64", "aarch64"):
        arch = "arm64"
    else:
        arch = machine
    
    return f"{system}-{arch}"

In [ ]:
# Test platform string
current = get_current_platform()
print(f"Current platform: {current}")

# Should be one of the expected formats
valid_prefixes = ["linux-", "darwin-", "win-"]
assert any(current.startswith(p) for p in valid_prefixes)

Current platform: linux-x64


## Path Utilities

Functions for cross-platform path handling, particularly for conda environments.

In [ ]:
#| export
def get_python_in_env(
    env_path: Path  # Path to conda environment root
) -> Path:  # Path to Python executable
    """Get the Python executable path for a conda environment.
    
    On Windows: env_path/python.exe
    On Unix: env_path/bin/python
    """
    if is_windows():
        return env_path / "python.exe"
    else:
        return env_path / "bin" / "python"

In [ ]:
# Test path generation
test_env = Path("/home/user/miniforge3/envs/test-env")
python_path = get_python_in_env(test_env)
print(f"Python path: {python_path}")

if is_windows():
    assert python_path.name == "python.exe"
else:
    assert "bin" in python_path.parts
    assert python_path.name == "python"

Python path: /home/user/miniforge3/envs/test-env/bin/python


## Process Management

Cross-platform utilities for subprocess creation and termination.

In [ ]:
#| export
def get_popen_isolation_kwargs() -> Dict[str, Any]:
    """Return kwargs for process isolation in subprocess.Popen.
    
    On Unix: Returns {'start_new_session': True}
    On Windows: Returns {'creationflags': CREATE_NEW_PROCESS_GROUP}
    
    Usage:
        process = subprocess.Popen(cmd, **get_popen_isolation_kwargs(), ...)
    """
    if is_windows():
        # CREATE_NEW_PROCESS_GROUP allows the process to be terminated
        # without affecting the parent process
        return {"creationflags": subprocess.CREATE_NEW_PROCESS_GROUP}
    else:
        # start_new_session creates a new process group on Unix
        return {"start_new_session": True}

In [ ]:
# Test isolation kwargs
kwargs = get_popen_isolation_kwargs()
print(f"Isolation kwargs: {kwargs}")

if is_windows():
    assert "creationflags" in kwargs
else:
    assert kwargs.get("start_new_session") == True

Isolation kwargs: {'start_new_session': True}


In [ ]:
#| export
def terminate_process(
    process: subprocess.Popen,  # Process to terminate
    timeout: float = 2.0  # Seconds to wait before force kill
) -> None:
    """Terminate a subprocess gracefully, with fallback to force kill.
    
    On all platforms:
    1. Calls process.terminate() (SIGTERM on Unix, TerminateProcess on Windows)
    2. Waits for timeout seconds
    3. If still running, calls process.kill() (SIGKILL on Unix, TerminateProcess on Windows)
    """
    if process is None or process.poll() is not None:
        return  # Already terminated
    
    process.terminate()
    try:
        process.wait(timeout=timeout)
    except subprocess.TimeoutExpired:
        process.kill()
        process.wait()  # Reap the process

In [ ]:
#| export
def terminate_self() -> None:
    """Terminate the current process (for worker suicide pact).
    
    On Unix: Sends SIGTERM to self for graceful shutdown
    On Windows: Calls os._exit() since Windows lacks SIGTERM
    """
    if is_windows():
        # Windows doesn't have SIGTERM, use os._exit for immediate termination
        # Exit code 1 indicates abnormal termination
        os._exit(1)
    else:
        import signal
        os.kill(os.getpid(), signal.SIGTERM)

## Shell Command Execution

Cross-platform shell command execution without hardcoded shell paths.

In [ ]:
#| export
def run_shell_command(
    cmd: str,  # Shell command to execute
    check: bool = True,  # Whether to raise on non-zero exit
    capture_output: bool = False,  # Whether to capture stdout/stderr
    **kwargs  # Additional kwargs passed to subprocess.run
) -> subprocess.CompletedProcess:
    """Run a shell command cross-platform.
    
    Unlike using shell=True with executable='/bin/bash', this function
    uses the platform's default shell:
    - Linux/macOS: /bin/sh (or $SHELL)
    - Windows: cmd.exe
    """
    print(f"Running: {cmd}")
    return subprocess.run(
        cmd,
        shell=True,
        check=check,
        capture_output=capture_output,
        **kwargs
    )

In [ ]:
#| export
def conda_env_exists(
    env_name: str,  # Name of the conda environment
    conda_cmd: str = "conda"  # Conda command (conda, mamba, micromamba)
) -> bool:
    """Check if a conda environment exists (cross-platform).
    
    Uses 'conda env list --json' instead of piping to grep,
    which doesn't work on Windows.
    """
    try:
        result = subprocess.run(
            [conda_cmd, "env", "list", "--json"],
            capture_output=True,
            text=True
        )
        if result.returncode != 0:
            return False
        
        data = json.loads(result.stdout)
        # Extract env names from paths
        for path in data.get('envs', []):
            if Path(path).name == env_name:
                return True
        return False
    except (subprocess.SubprocessError, json.JSONDecodeError, FileNotFoundError):
        return False

In [ ]:
# Test shell command (simple echo)
if is_windows():
    result = run_shell_command("echo hello", capture_output=True)
else:
    result = run_shell_command("echo hello", capture_output=True)

print(f"Return code: {result.returncode}")
print(f"Output: {result.stdout}")
assert result.returncode == 0

Running: echo hello
Return code: 0
Output: b'hello\n'


In [ ]:
# Test conda_env_exists (will return False for non-existent env)
exists = conda_env_exists("this-env-should-not-exist-12345")
print(f"Non-existent env exists: {exists}")
assert exists == False

# Test with base env "miniforge3" (should exist if miniforge3 is installed)
base_exists = conda_env_exists("miniforge3")
print(f"Base env exists: {base_exists}")

Non-existent env exists: False
Base env exists: True


## Conda/Micromamba Management

Functions for managing conda and micromamba runtimes, including downloading micromamba binaries and building commands with proper prefix handling for project-local mode.

In [ ]:
#| export
# Download URLs for micromamba binaries by platform
# These return .tar.bz2 archives that need extraction
MICROMAMBA_URLS: Dict[str, str] = {
    "linux-x64": "https://micro.mamba.pm/api/micromamba/linux-64/latest",
    "linux-arm64": "https://micro.mamba.pm/api/micromamba/linux-aarch64/latest",
    "darwin-x64": "https://micro.mamba.pm/api/micromamba/osx-64/latest",
    "darwin-arm64": "https://micro.mamba.pm/api/micromamba/osx-arm64/latest",
    "win-x64": "https://micro.mamba.pm/api/micromamba/win-64/latest",
}

In [ ]:
#| export
def get_micromamba_download_url(
    platform_str: Optional[str] = None  # Platform string (e.g., 'linux-x64'), uses current if None
) -> str:  # Download URL for micromamba binary
    """Get the micromamba download URL for the specified or current platform."""
    if platform_str is None:
        platform_str = get_current_platform()
    
    url = MICROMAMBA_URLS.get(platform_str)
    if url is None:
        raise ValueError(f"No micromamba download URL for platform: {platform_str}")
    
    return url

In [ ]:
#| export
def download_micromamba(
    dest_path: Path,  # Destination path for the micromamba binary
    platform_str: Optional[str] = None,  # Platform string, uses current if None
    show_progress: bool = True  # Whether to print progress messages
) -> bool:  # True if download succeeded
    """Download and extract micromamba binary to the specified path."""
    if platform_str is None:
        platform_str = get_current_platform()
    
    url = get_micromamba_download_url(platform_str)
    
    # Create parent directory if needed
    dest_path.parent.mkdir(parents=True, exist_ok=True)
    
    with tempfile.TemporaryDirectory() as tmpdir:
        tmpdir_path = Path(tmpdir)
        archive_path = tmpdir_path / "micromamba.tar.bz2"
        
        # Download the archive
        if show_progress:
            print(f"Downloading micromamba from {url}...")
        
        try:
            urllib.request.urlretrieve(url, archive_path)
        except urllib.error.URLError as e:
            if show_progress:
                print(f"Failed to download micromamba: {e}")
            return False
        
        # Extract the archive
        if show_progress:
            print("Extracting micromamba...")
        
        try:
            with tarfile.open(archive_path, "r:bz2") as tar:
                tar.extractall(tmpdir_path)
        except tarfile.TarError as e:
            if show_progress:
                print(f"Failed to extract archive: {e}")
            return False
        
        # Find the micromamba binary (usually at bin/micromamba or Library/bin/micromamba.exe)
        binary_name = "micromamba.exe" if is_windows() else "micromamba"
        extracted_binary = None
        
        for root, dirs, files in os.walk(tmpdir_path):
            if binary_name in files:
                extracted_binary = Path(root) / binary_name
                break
        
        if extracted_binary is None:
            if show_progress:
                print(f"Could not find {binary_name} in extracted archive")
            return False
        
        # Move to destination
        shutil.copy2(extracted_binary, dest_path)
        
        # Make executable on Unix
        if not is_windows():
            dest_path.chmod(dest_path.stat().st_mode | 0o755)
        
        if show_progress:
            print(f"Micromamba installed to {dest_path}")
        
        return True

In [ ]:
# Test micromamba URL functions
current_platform = get_current_platform()
url = get_micromamba_download_url()
print(f"Platform: {current_platform}")
print(f"Download URL: {url}")

# Verify all platforms have URLs
for plat in ["linux-x64", "linux-arm64", "darwin-x64", "darwin-arm64", "win-x64"]:
    assert plat in MICROMAMBA_URLS, f"Missing URL for {plat}"
    print(f"  {plat}: {MICROMAMBA_URLS[plat]}")

Platform: linux-x64
Download URL: https://micro.mamba.pm/api/micromamba/linux-64/latest
  linux-x64: https://micro.mamba.pm/api/micromamba/linux-64/latest
  linux-arm64: https://micro.mamba.pm/api/micromamba/linux-aarch64/latest
  darwin-x64: https://micro.mamba.pm/api/micromamba/osx-64/latest
  darwin-arm64: https://micro.mamba.pm/api/micromamba/osx-arm64/latest
  win-x64: https://micro.mamba.pm/api/micromamba/win-64/latest


### Conda Command Building

Functions to build conda/mamba/micromamba commands with proper prefix handling for project-local mode.

In [ ]:
#| export
def get_conda_command(
    config: CJMConfig  # Configuration object with runtime settings
) -> List[str]:  # Base command with prefix args if needed
    """Get the conda/mamba/micromamba base command with prefix args for local mode."""
    # Late import to avoid circular dependency
    from cjm_plugin_system.core.config import CondaType, RuntimeMode
    
    if config.runtime.conda_type == CondaType.MICROMAMBA:
        # Get binary path from config or use default
        platform_key = get_current_platform()
        if config.runtime.binaries and platform_key in config.runtime.binaries:
            binary = str(config.runtime.binaries[platform_key])
        else:
            binary = "micromamba"
        
        # Add root prefix for local mode
        if config.runtime.mode == RuntimeMode.LOCAL and config.runtime.prefix:
            return [binary, "-r", str(config.runtime.prefix)]
        return [binary]
    
    elif config.runtime.conda_type == CondaType.MINIFORGE:
        return ["mamba"]
    
    else:  # CondaType.CONDA or default
        return ["conda"]

In [ ]:
#| export
def build_conda_command(
    config: CJMConfig,  # Configuration object with runtime settings
    *args: str  # Additional command arguments
) -> List[str]:  # Complete command ready for subprocess
    """Build a complete conda/mamba/micromamba command."""
    base = get_conda_command(config)
    return base + list(args)

In [ ]:
#| export
def get_micromamba_binary_path(
    config: CJMConfig  # Configuration object with runtime settings
) -> Optional[Path]:  # Path to micromamba binary or None
    """Get the configured micromamba binary path for the current platform."""
    platform_key = get_current_platform()
    
    if config.runtime.binaries and platform_key in config.runtime.binaries:
        return config.runtime.binaries[platform_key]
    
    # Default location if prefix is set
    if config.runtime.prefix:
        binary_name = "micromamba.exe" if is_windows() else "micromamba"
        return config.runtime.prefix / "bin" / binary_name
    
    return None

In [ ]:
#| export
def ensure_runtime_available(
    config: CJMConfig  # Configuration object with runtime settings
) -> bool:  # True if runtime is available
    """Check if the configured conda/micromamba runtime is available."""
    # Late import to avoid circular dependency
    from cjm_plugin_system.core.config import CondaType
    
    if config.runtime.conda_type == CondaType.MICROMAMBA:
        binary_path = get_micromamba_binary_path(config)
        if binary_path and binary_path.exists():
            return True
        # Also check if micromamba is in PATH
        try:
            result = subprocess.run(
                ["micromamba", "--version"],
                capture_output=True,
                text=True
            )
            return result.returncode == 0
        except FileNotFoundError:
            return False
    
    elif config.runtime.conda_type == CondaType.MINIFORGE:
        try:
            result = subprocess.run(
                ["mamba", "--version"],
                capture_output=True,
                text=True
            )
            return result.returncode == 0
        except FileNotFoundError:
            return False
    
    else:  # CondaType.CONDA
        try:
            result = subprocess.run(
                ["conda", "--version"],
                capture_output=True,
                text=True
            )
            return result.returncode == 0
        except FileNotFoundError:
            return False

In [ ]:
# Test conda command building
from cjm_plugin_system.core.config import CJMConfig, RuntimeConfig, CondaType, RuntimeMode

# Test with default conda
default_config = CJMConfig()
cmd = get_conda_command(default_config)
print(f"Default (conda): {cmd}")
assert cmd == ["conda"]

# Test with miniforge/mamba
mamba_config = CJMConfig(runtime=RuntimeConfig(conda_type=CondaType.MINIFORGE))
cmd = get_conda_command(mamba_config)
print(f"Miniforge (mamba): {cmd}")
assert cmd == ["mamba"]

# Test with micromamba system mode
micromamba_system = CJMConfig(runtime=RuntimeConfig(conda_type=CondaType.MICROMAMBA))
cmd = get_conda_command(micromamba_system)
print(f"Micromamba (system): {cmd}")
assert cmd == ["micromamba"]

# Test with micromamba local mode
micromamba_local = CJMConfig(
    runtime=RuntimeConfig(
        conda_type=CondaType.MICROMAMBA,
        mode=RuntimeMode.LOCAL,
        prefix=Path("./runtime")
    )
)
cmd = get_conda_command(micromamba_local)
print(f"Micromamba (local): {cmd}")
# Path("./runtime") normalizes to "runtime" when converted to str
assert cmd == ["micromamba", "-r", "runtime"]

# Test build_conda_command
full_cmd = build_conda_command(micromamba_local, "create", "-n", "test-env", "-y")
print(f"Full command: {full_cmd}")
assert full_cmd == ["micromamba", "-r", "runtime", "create", "-n", "test-env", "-y"]

Default (conda): ['conda']
Miniforge (mamba): ['mamba']
Micromamba (system): ['micromamba']
Micromamba (local): ['micromamba', '-r', 'runtime']
Full command: ['micromamba', '-r', 'runtime', 'create', '-n', 'test-env', '-y']


## Summary

This module provides the following cross-platform utilities:

### Detection
- `is_windows()`, `is_macos()`, `is_linux()` - OS detection
- `is_apple_silicon()` - Apple Silicon detection for MPS
- `get_current_platform()` - Platform string like "linux-x64"

### Paths
- `get_python_in_env(env_path)` - Python executable path in conda env

### Process Management
- `get_popen_isolation_kwargs()` - Kwargs for subprocess isolation
- `terminate_process(process, timeout)` - Graceful process termination
- `terminate_self()` - Self-termination for suicide pact

### Shell Commands
- `run_shell_command(cmd, ...)` - Cross-platform shell execution
- `conda_env_exists(env_name)` - Check if conda env exists

### Conda/Micromamba Management
- `MICROMAMBA_URLS` - Download URLs for micromamba binaries by platform
- `get_micromamba_download_url(platform)` - Get download URL for current/specified platform
- `download_micromamba(dest_path)` - Download and extract micromamba binary
- `get_conda_command(config)` - Get base command with prefix args for local mode
- `build_conda_command(config, *args)` - Build complete conda/micromamba command
- `get_micromamba_binary_path(config)` - Get configured binary path for current platform
- `ensure_runtime_available(config)` - Check if runtime is available

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()